In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.getOrCreate()

22/05/25 22:12:22 WARN Utils: Your hostname, hinzles-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.86.244 instead (on interface en0)
22/05/25 22:12:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/25 22:12:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


This exercises uses the case.csv, dept.csv, and source.csv files from the san antonio 311 call dataset.

# 1.) Read the case, department, and source data into their own spark dataframes.

In [4]:
case = spark.read.csv("/Users/hinzlehome/codeup-data-science/spark-exercises/csv/case.csv", sep=",", header=True, inferSchema=True)

In [5]:
case.show(1, vertical=True, truncate=False)

-RECORD 0----------------------------------------------------
 case_id              | 1014127332                           
 case_opened_date     | 1/1/18 0:42                          
 case_closed_date     | 1/1/18 12:29                         
 SLA_due_date         | 9/26/20 0:42                         
 case_late            | NO                                   
 num_days_late        | -998.5087616000001                   
 case_closed          | YES                                  
 dept_division        | Field Operations                     
 service_request_type | Stray Animal                         
 SLA_days             | 999.0                                
 case_status          | Closed                               
 source_id            | svcCRMLS                             
 request_address      | 2315  EL PASO ST, San Antonio, 78207 
 council_district     | 5                                    
only showing top 1 row



In [7]:
dpt = spark.read.csv("/Users/hinzlehome/codeup-data-science/spark-exercises/csv/dept.csv", sep=",", header=True, inferSchema=True)

In [8]:
dpt.show(1, vertical=True, truncate=False)

-RECORD 0----------------------------------
 dept_division          | 311 Call Center  
 dept_name              | Customer Service 
 standardized_dept_name | Customer Service 
 dept_subject_to_SLA    | YES              
only showing top 1 row



In [10]:
source = spark.read.csv("/Users/hinzlehome/codeup-data-science/spark-exercises/csv/source.csv", sep=",", header=True, inferSchema=True)

In [11]:
source.show(1, vertical = True, truncate=False)

-RECORD 0---------------------------
 source_id       | 100137           
 source_username | Merlene Blodgett 
only showing top 1 row



# 2.) Let's see how writing to the local disk works in spark:

Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json

In [12]:
source.write.json("data/sources_json", mode="overwrite")

In [13]:
source.write.csv("data/sources_csv", mode="overwrite")

Inspect your folder structure. What do you notice?

In [14]:
#In each folder, there is a success text file and the actual json or csv file

# 3.) Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [15]:
case.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)



In [16]:
dpt.printSchema()

root
 |-- dept_division: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: string (nullable = true)



In [17]:
source.printSchema()

root
 |-- source_id: string (nullable = true)
 |-- source_username: string (nullable = true)



---

# 1.) How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [18]:
fmt = "M/d/yy H:mm"
case = (
    case.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_closed_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_due_date", to_timestamp("case_opened_date", fmt))
    .withColumn("SLA_due_date", to_timestamp("SLA_due_date", fmt))
)

In [19]:
open_cases = (
    case.filter(expr('case_closed == "NO"'))
    .withColumn('days_past_SLA', datediff(current_timestamp(),'SLA_due_date'))
    .withColumn('duration', datediff(current_timestamp(),'case_opened_date'))
)

In [20]:
#most recent 
open_cases.select(min('days_past_SLA')).show()

+------------------+
|min(days_past_SLA)|
+------------------+
|                -2|
+------------------+



In [21]:
#oldest duration
open_cases.select(max('duration')).show()

+-------------+
|max(duration)|
+-------------+
|         1970|
+-------------+



# 2.) How many Stray Animal cases are there?

In [22]:
case.where(expr(
    'service_request_type=="Stray Animal"')).count()

26760

# 3.) How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [23]:
#dept division is field operations and service request type is not officer standby
(case.where(expr('dept_division == "Field Operations"'))
.where(expr('service_request_type != "Officer Standby"'))).count()

113902

# 4.) Convert the council_district column to a string column.

In [24]:
case.select(case.council_district.cast('string')).show(5)

+----------------+
|council_district|
+----------------+
|               5|
|               3|
|               3|
|               3|
|               7|
+----------------+
only showing top 5 rows



In [25]:
#case = case.council_district.cast('string')

In [26]:
case = case.withColumn("council_district",col("council_district").cast("string"))

In [27]:
case.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- SLA_due_date: timestamp (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = true)
 |-- case_due_date: timestamp (nullable = true)



# 5.) Extract the year from the case_closed_date column.

In [28]:
year = case.withColumn('year', year('case_closed_date'))

In [29]:
year.select('year').show()

+----+
|year|
+----+
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
+----+
only showing top 20 rows



# 6.) Convert num_days_late from days to hours in new columns num_hours_late.

In [30]:
case = case.withColumn(
    "num_hours_late", expr("num_days_late * 24 AS num_hours_late")
)

case.select("num_days_late", "num_hours_late").show(5)

+-------------------+-------------------+
|      num_days_late|     num_hours_late|
+-------------------+-------------------+
| -998.5087616000001|     -23964.2102784|
|-2.0126041669999997|-48.302500007999996|
|       -3.022337963|      -72.536111112|
|       -15.01148148|      -360.27555552|
|0.37216435200000003|  8.931944448000001|
+-------------------+-------------------+
only showing top 5 rows



# 7.) Join the case data with the source and department data.

In [31]:
df = case.join(source, on=case.source_id == source.source_id, how = 'left').drop(source.source_id)

In [32]:
df = df.join(dpt, on=df.dept_division == dpt.dept_division, how = 'left').drop(dpt.dept_division)

In [33]:
df.show(2, vertical = True)

-RECORD 0--------------------------------------
 case_id                | 1014127332           
 case_opened_date       | 2018-01-01 00:42:00  
 case_closed_date       | 2018-01-01 00:42:00  
 SLA_due_date           | 2020-09-26 00:42:00  
 case_late              | NO                   
 num_days_late          | -998.5087616000001   
 case_closed            | YES                  
 dept_division          | Field Operations     
 service_request_type   | Stray Animal         
 SLA_days               | 999.0                
 case_status            | Closed               
 source_id              | svcCRMLS             
 request_address        | 2315  EL PASO ST,... 
 council_district       | 5                    
 case_due_date          | 2018-01-01 00:42:00  
 num_hours_late         | -23964.2102784       
 source_username        | svcCRMLS             
 dept_name              | Animal Care Services 
 standardized_dept_name | Animal Care Services 
 dept_subject_to_SLA    | YES           

# 8.) Are there any cases that do not have a request source?

In [34]:
#no nulls
df.filter("source_username is null").show(vertical=True)

(0 rows)



# 9.) What are the top 10 service request types in terms of number of requests?

In [35]:
df.groupby('service_request_type').count().sort(col('count').desc()).show(10)

+--------------------+-----+
|service_request_type|count|
+--------------------+-----+
|           No Pickup|89210|
|Overgrown Yard/Trash|66403|
|        Bandit Signs|32968|
|        Damaged Cart|31163|
|Front Or Side Yar...|28920|
|        Stray Animal|27361|
|Aggressive Animal...|25492|
|Cart Exchange Req...|22608|
|Junk Vehicle On P...|21649|
|     Pot Hole Repair|20827|
+--------------------+-----+
only showing top 10 rows



# 10.) What are the top 10 service request types in terms of average days late?

In [36]:
df.show(1, False, True)

-RECORD 0------------------------------------------------------
 case_id                | 1014127332                           
 case_opened_date       | 2018-01-01 00:42:00                  
 case_closed_date       | 2018-01-01 00:42:00                  
 SLA_due_date           | 2020-09-26 00:42:00                  
 case_late              | NO                                   
 num_days_late          | -998.5087616000001                   
 case_closed            | YES                                  
 dept_division          | Field Operations                     
 service_request_type   | Stray Animal                         
 SLA_days               | 999.0                                
 case_status            | Closed                               
 source_id              | svcCRMLS                             
 request_address        | 2315  EL PASO ST, San Antonio, 78207 
 council_district       | 5                                    
 case_due_date          | 2018-01-01 00:

In [37]:
df.where('case_late=="YES"')\
.groupBy('service_request_type')\
.agg(mean('num_days_late'))\
.sort(desc('avg(num_days_late)'))\
.show(10, truncate=False)

+--------------------------------------+------------------+
|service_request_type                  |avg(num_days_late)|
+--------------------------------------+------------------+
|Zoning: Recycle Yard                  |210.89201994318182|
|Zoning: Junk Yards                    |200.2051760849428 |
|Structure/Housing Maintenance         |190.20707698509807|
|Donation Container Enforcement        |171.09115313942624|
|Storage of Used Mattress              |163.96812829714287|
|Labeling for Used Mattress            |162.43032902285717|
|Record Keeping of Used Mattresses     |153.99724039428568|
|Signage Requied for Sale of Used Mattr|151.63868055333333|
|Traffic Signal Graffiti               |137.64583330000002|
|License Requied Used Mattress Sales   |128.79828704142858|
+--------------------------------------+------------------+
only showing top 10 rows



# 11.) Does number of days late depend on department?

In [38]:
df.where('case_late=="YES"')\
.groupBy('standardized_dept_name')\
.agg(mean('num_days_late'))\
.sort(desc('avg(num_days_late)'))\
.show(truncate=False)

+------------------------+------------------+
|standardized_dept_name  |avg(num_days_late)|
+------------------------+------------------+
|Customer Service        |88.18248182589826 |
|DSD/Code Enforcement    |49.506339986350525|
|Animal Care Services    |23.446729634738205|
|Parks & Recreation      |22.42780719272415 |
|Trans & Cap Improvements|10.662950455078676|
|Solid Waste             |7.1471727895574135|
|Metro Health            |6.494699602827868 |
+------------------------+------------------+



# 12.) How do number of days late depend on department and request type?

In [39]:
df.where('case_late=="YES"')\
.groupBy('standardized_dept_name', 'service_request_type')\
.agg(mean('num_days_late'))\
.sort(desc('avg(num_days_late)'))\
.show(truncate=False)

+------------------------+----------------------------------------+------------------+
|standardized_dept_name  |service_request_type                    |avg(num_days_late)|
+------------------------+----------------------------------------+------------------+
|DSD/Code Enforcement    |Zoning: Recycle Yard                    |210.89201994318182|
|DSD/Code Enforcement    |Zoning: Junk Yards                      |200.2051760849428 |
|DSD/Code Enforcement    |Structure/Housing Maintenance           |190.20707698509807|
|DSD/Code Enforcement    |Donation Container Enforcement          |171.09115313942624|
|DSD/Code Enforcement    |Storage of Used Mattress                |163.96812829714287|
|DSD/Code Enforcement    |Labeling for Used Mattress              |162.43032902285717|
|DSD/Code Enforcement    |Record Keeping of Used Mattresses       |153.99724039428568|
|DSD/Code Enforcement    |Signage Requied for Sale of Used Mattr  |151.63868055333333|
|Trans & Cap Improvements|Traffic Signal Gr